<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#выгружаем-датасет:" data-toc-modified-id="выгружаем-датасет:-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>выгружаем датасет:</a></span></li></ul></div>

Задание 1: Реализовать Bag of words & tfidf на jokes.csv (в архиве)  



In [1]:
!pip install --upgrade nltk gensim bokeh umap-learn

import itertools
import string

import numpy as np
import umap
from nltk.tokenize import WordPunctTokenizer

from matplotlib import pyplot as plt

from IPython.display import clear_output

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()




  Using cached bokeh-3.1.0-py3-none-any.whl (8.3 MB)
  Attempting uninstall: bokeh
    Found existing installation: bokeh 2.4.3
    Uninstalling bokeh-2.4.3:
      Successfully uninstalled bokeh-2.4.3


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vasilijpancenko/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/vasilijpancenko/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Готовые модели

In [3]:
# import gensim
import gensim.downloader as api 
model = api.load('glove-twitter-25') 
model.most_similar(positive=["coder", "money"], negative=["brain"])

[==================================================] 100.0% 104.8/104.8MB downloaded


[('realtor', 0.8265186548233032),
 ('gfx', 0.8249695897102356),
 ('caterers', 0.798202395439148),
 ('beatmaker', 0.7936854362487793),
 ('recruiter', 0.7892400026321411),
 ('sfi', 0.784467339515686),
 ('sosh', 0.7840632796287537),
 ('promoter', 0.7838250994682312),
 ('smallbusiness', 0.7786215543746948),
 ('promoters', 0.77646803855896)]

In [4]:
import bs4 as bs
import urllib.request
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')

scrapped_data = urllib.request.urlopen('https://en.wikipedia.org/wiki/Artificial_intelligence')
article = scrapped_data.read()

parsed_article = bs.BeautifulSoup(article,'lxml')

paragraphs = parsed_article.find_all('p')

article_text = ""

for p in paragraphs:
    article_text += p.text

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vasilijpancenko/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vasilijpancenko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Cleaing the text
processed_article = article_text.lower()
processed_article = re.sub('[^a-zA-Z]', ' ', processed_article )
processed_article = re.sub(r'\s+', ' ', processed_article)

# Preparing the dataset
all_sentences = nltk.sent_tokenize(processed_article)

all_words = [nltk.word_tokenize(sent) for sent in all_sentences]

# Removing Stop Words
from nltk.corpus import stopwords
for i in range(len(all_words)):
    all_words[i] = [w for w in all_words[i] if w not in stopwords.words('english')]


In [6]:
from gensim.models import Word2Vec
word2vec = Word2Vec(all_words, min_count=2)

In [9]:
vocabulary = word2vec.wv.vocab
print(vocabulary)

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [10]:
v1 = word2vec.wv['artificial']

In [11]:
sim_words = word2vec.wv.most_similar('intelligence')
sim_words

[('training', 0.40299665927886963),
 ('ai', 0.37336066365242004),
 ('general', 0.36903053522109985),
 ('would', 0.35330915451049805),
 ('model', 0.33899199962615967),
 ('systems', 0.33775126934051514),
 ('approaches', 0.3296746611595154),
 ('also', 0.3175889551639557),
 ('machine', 0.313571959733963),
 ('easy', 0.30880311131477356)]

Задание 2: на базе датасета обучить модель 

# выгружаем датасет:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc